# Notebook: Práctica iterativa (simple y didáctica)

Esta notebook te guía paso a paso. Cambia **solo** las variables `catalog` y `schema` para decidir dónde se crean las tablas.

**Tablas que se crearán**:
- Entrada: `iris_input`
- Intermedia: `iris_intermediate`
- Final: `iris_predictions`


In [ ]:
# 1) Preparar el entorno: instala dependencias mínimas
%pip install pandas scikit-learn mlflow

In [ ]:
# 2) Configura tu espacio (cámbialo por el tuyo)
catalog = "demo_uc"   # p. ej., "main" o tu catálogo
schema  = "validacion" # p. ej., "sandbox" o tu esquema

table_input       = f"{catalog}.{schema}.iris_input"
table_intermediate= f"{catalog}.{schema}.iris_intermediate"
table_predictions = f"{catalog}.{schema}.iris_predictions"

print("Usando:")
print("  Entrada     :", table_input)
print("  Intermedia  :", table_intermediate)
print("  Predicciones:", table_predictions)

## 3) Cargar dataset público y crear tabla de **entrada**

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv"
df = pd.read_csv(url)

spark_df = spark.createDataFrame(df)
spark_df.write.mode("overwrite").saveAsTable(table_input)
display(spark.table(table_input))

## 4) Crear tabla **intermedia** (transformación simple)
En un caso real harías varias transformaciones; aquí solo añadimos una columna sencilla.

In [ ]:
df_intermediate = df.copy()
df_intermediate["petal_area"] = df_intermediate["petal_length"] * df_intermediate["petal_width"]
spark.createDataFrame(df_intermediate).write.mode("overwrite").saveAsTable(table_intermediate)
display(spark.table(table_intermediate))

## 5) Entrenar un modelo muy sencillo (clasificación)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import mlflow.sklearn

X = df_intermediate.drop(columns=["species"])  # features
y = df_intermediate["species"]                 # etiqueta

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

acc = accuracy_score(y_test, model.predict(X_test))
print(f"Exactitud del modelo: {acc:.2f}")

# Registro de ejemplo en MLflow
mlflow.sklearn.log_model(model, "modelo_logistico_demo")

## 6) Guardar tabla **final** con predicciones

In [ ]:
preds = model.predict(X_test)
df_preds = pd.DataFrame({
    "real": y_test.values,
    "prediccion": preds
})
spark.createDataFrame(df_preds).write.mode("overwrite").saveAsTable(table_predictions)
display(spark.table(table_predictions))

## 7) (Opcional) Comparar con otras tablas de otros equipos
Si cuentas con acceso de solo lectura a otros catálogos/esquemas, puedes comparar resultados con un `JOIN`.
Ejemplo SQL (ajusta nombres reales):
```sql
SELECT a.*, b.prediccion AS pred_otro
FROM demo_uc.validacion.iris_predictions a
LEFT JOIN prod_uc.linea1.iris_predictions b
  ON a.real = b.real
```
